In [1]:
import pandas as pd
import numpy as np
import glob

In [4]:
filteredGeneList = pd.read_csv('../RBC/Database/filteredGenesDetails_human_240118.txt',sep='\t')
acc2gene={}
gene2acc={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
    gene2acc[row['GeneName']] = row['AccNum']
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [25]:
TIS = pd.read_csv('../RBC/Database/accnum2tis.csv')
TIS.columns = ['AccNum','TIS']
TIS.head()

,AccNum,TIS
0,NM_017582,101.0
1,NM_001276267,111.0
2,NM_173469,79.0
3,NM_001330474,34.0
4,NM_014372,102.0


In [7]:
logL_L24 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_L24_t1_enriched_likelihood.csv')
logL_S15_t0 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_S15_t0_enriched_likelihood.csv')
logL_S15_t1 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_S15_t1_enriched_likelihood.csv')
logL_L24.head(2)

,Unnamed: 0,logLikelihood,Gene,AccNum
0,0,-14.542486,MYO1D,NM_015194
1,1,-14.550683,PPP1R14B,NM_138689


In [30]:
mfe_fp = pd.read_csv('../RBC/MOTIF/rnafold/fpUTR_minE_normbyLen_fillmean.csv')
mfe_fp_orf = pd.read_csv('../RBC/MOTIF/rnafold/utr_and_orf_minE_normbyLen.csv')
mfe_fp.columns = ['AccNum','MFE_fpUTR_normed']
mfe_fp_orf.columns = ['AccNum','MFE_fpUTRandORF_normed']

In [31]:
comb = pd.DataFrame(logL_L24.loc[:,'Gene'])
comb['AccNum'] = logL_L24['AccNum']
comb['seqlogo_logL_L24_t1'] = logL_L24['logLikelihood']
comb['seqlogo_logL_S15_t0'] = logL_S15_t0['logLikelihood']
comb['seqlogo_logL_S15_t1'] = logL_S15_t1['logLikelihood']
comb = pd.merge(comb, TIS, how='inner', on=['AccNum'])
comb = pd.merge(comb, mfe_fp, how='inner', on=['AccNum'])
comb = pd.merge(comb, mfe_fp_orf, how='inner', on=['AccNum'])

In [32]:
comb

,Gene,AccNum,seqlogo_logL_L24_t1,seqlogo_logL_S15_t0,seqlogo_logL_S15_t1,TIS,MFE_fpUTR_normed,MFE_fpUTRandORF_normed
0,MYO1D,NM_015194,-14.542486,-15.117326,-15.447782,91.0,-0.522658,-0.504090
1,PPP1R14B,NM_138689,-14.550683,-15.397989,-15.667537,91.0,-0.644104,-0.622038
2,AGFG1,NM_001135187,-14.663012,-15.412761,-15.725220,113.0,-0.666440,-0.598229
3,LAMP1,NM_005561,-14.668466,-15.438871,-15.744788,98.0,-0.519715,-0.509029
4,AFG3L2,NM_006796,-14.691183,-15.474773,-15.753664,113.0,-0.560619,-0.620646
...,...,...,...,...,...,...,...,...
7885,SDCBP,NM_001348341,-33.683685,-29.864479,-28.686124,44.0,-0.312993,-0.308519
7886,RPRD1A,NM_001303411,-33.846032,-29.997533,-28.854125,79.0,-0.464314,-0.414923
7887,FASTKD1,NM_001322046,-33.846032,-30.293971,-29.278236,73.0,-0.330131,-0.329384
7888,SCRN3,NM_024583,-33.975719,-31.335351,-29.994056,67.0,-0.305612,-0.258283


In [33]:
comb.to_csv('collate_genes.csv')